In [56]:
import inspect
import torch

classes = set([])
ignored_modules = ['onnx', 'jit', 'ao', 'amp', 'autograd',
                   'utils', 'return_types', 'special', 'backends',
                   'nn.intrinsic']
added_builtins_classes = 'Tensor'
ignored_functions = []

def is_correct_module(module, name, prev_module, prevs):
    return not name.startswith('_') \
           and not name.endswith('_') \
           and not getattr(module, '__file__', '').startswith('_') \
           and 'torch' in getattr(module, '__file__', '') \
           and not getattr(module, '__file__', '') in prevs \
           and prev_module.__name__ in module.__name__ \
           and module.__name__[module.__name__.find('.') + 1:] not in ignored_modules


def is_correct_func(name):
    return not name.startswith('_') \
           and not name.endswith('_')


def is_correct_class(obj, prev):
    return not obj.__name__.startswith('_') \
           and not obj.__name__.endswith('_') \
           and not obj.__module__ + '.' + obj.__name__ in classes \
           and obj.__module__ == prev.__name__


def get_args(func):
    args, varargs, keywords, defaults, _, _, _ = inspect.getfullargspec(func)
    return args, defaults


def add_param(name, obj, hierarchy, prev, prevs, num):
    if inspect.ismodule(obj):
        if is_correct_module(obj, name, prev, prevs):
            print('\t'*num, obj.__name__)
            new_prevs = prevs.copy()
            new_prevs.append(obj.__file__)
            hierarchy[name] = {}
            for n, o in inspect.getmembers(obj):
                add_param(n, o, hierarchy[name], obj, new_prevs, num+1)
    elif inspect.isclass(obj):
        if is_correct_class(obj, prev):
            print('\t'*num, name)
            hierarchy[name] = {}
            classes.add(obj.__module__ + '.' + obj.__name__)
            # for n, o in inspect.getmembers(obj):
            #     add_param(n, o, hierarchy[name], obj, prevs, num + 1)
    elif inspect.isfunction(obj) or inspect.ismethod(obj) or inspect.isbuiltin(obj):
        if is_correct_func(name):
            print('\t'*num, name)
            arg_dict = None
            if not inspect.isbuiltin(obj):
                arg_dict = {}
                args, defaults = get_args(obj)
                if args:
                    for i, arg in enumerate(args):
                        arg_dict[arg] = defaults[i] if defaults and len(defaults) > i else None
            torch_hierarchy[name] = arg_dict
    

torch_hierarchy = {}
for name, obj in inspect.getmembers(torch):
    prevs = [torch.__file__]
    if inspect.ismodule(obj):
        if is_correct_module(obj, name, torch, prevs):
            print(name)
            prevs.append(obj.__file__)
            torch_hierarchy[name] = {}
            for n, o in inspect.getmembers(obj):
                add_param(n, o, torch_hierarchy[name], obj, prevs, 1)
    elif inspect.isclass(obj):
        if is_correct_class(obj, torch) and added_builtins_classes in name:
            print(name)
            classes.add(obj.__module__ + '.' + obj.__name__)
            torch_hierarchy[name] = {}
            # for n, o in inspect.getmembers(obj):
            #     add_param(n, o, torch_hierarchy[name], obj, prevs, 1)
    elif inspect.isfunction(obj) or inspect.ismethod(obj) or inspect.isbuiltin(obj):
        if is_correct_func(name):
            print(name)
            arg_dict = None
            if not inspect.isbuiltin(obj):
                arg_dict = {}
                args, defaults = get_args(obj)
                if args:
                    for i, arg in enumerate(args):
                        arg_dict[arg] = defaults[i] if defaults and len(defaults) > i else None
            torch_hierarchy[name] = arg_dict

BFloat16Tensor
BoolTensor
ByteTensor
CharTensor
DoubleTensor
FloatTensor
HalfTensor
IntTensor
LongTensor
ShortTensor
Tensor
TensorType
abs
absolute
acos
acosh
adaptive_avg_pool1d
adaptive_max_pool1d
add
addbmm
addcdiv
addcmul
addmm
addmv
addr
adjoint
affine_grid_generator
alias_copy
align_tensors
all
allclose
alpha_dropout
amax
amin
aminmax
angle
any
arange
arccos
arccosh
arcsin
arcsinh
arctan
arctan2
arctanh
are_deterministic_algorithms_enabled
argmax
argmin
argsort
argwhere
as_strided
as_strided_copy
as_strided_scatter
as_tensor
asarray
asin
asinh
atan
atan2
atanh
atleast_1d
atleast_2d
atleast_3d
autocast_decrement_nesting
autocast_increment_nesting
avg_pool1d
baddbmm
bartlett_window
batch_norm
batch_norm_backward_elemt
batch_norm_backward_reduce
batch_norm_elemt
batch_norm_gather_stats
batch_norm_gather_stats_with_counts
batch_norm_stats
batch_norm_update_stats
bernoulli
bilinear
binary_cross_entropy_with_logits
bincount
binomial
bitwise_and
bitwise_left_shift
bitwise_not
bitwise_or

c:\Users\kozlo\anaconda3\lib\site-packages\torch\distributed\distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [57]:
torch_hierarchy

{'BFloat16Tensor': {},
 'BoolTensor': {},
 'ByteTensor': {},
 'CharTensor': {},
 'DoubleTensor': {},
 'FloatTensor': {},
 'HalfTensor': {},
 'IntTensor': {},
 'LongTensor': {},
 'ShortTensor': {},
 'Tensor': {},
 'TensorType': {},
 'abs': None,
 'absolute': None,
 'acos': None,
 'acosh': None,
 'adaptive_avg_pool1d': None,
 'adaptive_max_pool1d': {},
 'add': None,
 'addbmm': None,
 'addcdiv': None,
 'addcmul': None,
 'addmm': None,
 'addmv': None,
 'addr': None,
 'adjoint': None,
 'affine_grid_generator': None,
 'alias_copy': None,
 'align_tensors': {},
 'all': None,
 'allclose': None,
 'alpha_dropout': {'input': 0.5,
  'p': False,
  'training': False,
  'inplace': None},
 'amax': {'input': None, 'dim': None},
 'amin': {'input': None, 'dim': None},
 'aminmax': None,
 'angle': None,
 'any': None,
 'arange': None,
 'arccos': None,
 'arccosh': None,
 'arcsin': None,
 'arcsinh': None,
 'arctan': None,
 'arctan2': None,
 'arctanh': None,
 'are_deterministic_algorithms_enabled': {},
 'argmax